In [1]:
from elo import set_elo_df, set_tilts
from elo_update import update_elo_with_fixtures
from fetch_elo import fetch_elo_data
from fixtures import compute_initial_tilts, get_fixtures
from simulation import simulate_match, simulate_season, build_season_summary
from table import build_league_table

/Users/thorjan/git/elo/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Load data

In [2]:
fixtures = get_fixtures(force_refresh=False)  # Set to True to force refresh from API
elo = fetch_elo_data(force_refresh=False)  # Set to True to force refresh from ClubELO API
set_elo_df(elo) # Set the elo_df globally for Club class
tilts = compute_initial_tilts(fixtures)
set_tilts(tilts)  # Set the tilts globally for Club class

Loading fixtures from cache: fixtures.parquet
Loading ELO data from cache: elo_latest.parquet


Update ELO for missing matches

In [3]:
elo = update_elo_with_fixtures(elo, fixtures, tilts=tilts)
set_elo_df(elo) # Set the elo_df globally for Club class

6 teams had their ELO updated.


# Current standings

In [4]:
build_league_table(fixtures.loc[fixtures.season == 2025])

,Position,Team,Games,Wins,Draws,Losses,Goals,GD,Points,GF,GA
0,1,Viking,19,13,3,3,47-26,+21,42,47,26
1,2,Bodø/Glimt,18,12,3,3,44-17,+27,39,44,17
2,3,Brann,17,10,3,4,31-25,+6,33,31,25
3,4,Tromsø,17,10,3,4,29-23,+6,33,29,23
4,5,Rosenborg,18,8,6,4,24-21,+3,30,24,21
5,6,Sandefjord,17,9,0,8,33-24,+9,27,33,24
6,7,Fredrikstad,18,7,5,6,23-20,+3,26,23,20
7,8,KFUM Oslo,17,7,4,6,29-20,+9,25,29,20
8,9,Molde,17,7,2,8,25-23,+2,23,25,23
9,10,Sarpsborg 08,17,5,7,5,28-24,+4,22,28,24


# Simulation

In [5]:
iterations = 300

stats_tracker_season, position_counts_season = simulate_season(fixtures, n_simulations=iterations)
table_mean_season, position_probs_season = build_season_summary(stats_tracker_season, position_counts_season, use_median=False)
table_median_season, _ = build_season_summary(stats_tracker_season, position_counts_season, use_median=True)

139 games have been played. Starting 300 simulations of 101 games.


Simulating seasons:   0%|          | 0/300 [00:00<?, ?it/s]

Mean

In [6]:
display(table_mean_season)
display(position_probs_season)

,Position,Team,Games,Exp Wins,Exp Draws,Exp Losses,Goals,Exp Points
0,1,Bodø/Glimt,30.0,21.05,4.82,4.13,73.83-25.22,67.96
1,2,Viking,30.0,20.08,5.05,4.87,72.17-37.55,65.30
2,3,Brann,30.0,17.08,5.68,7.24,55.01-40.1,56.93
3,4,Tromsø,30.0,16.31,6.11,7.58,49.05-37.21,55.03
4,5,Rosenborg,30.0,13.75,8.55,7.70,43.36-35.69,49.79
5,6,Molde,30.0,13.03,4.66,12.31,46.67-40.91,43.75
6,7,Sandefjord,30.0,13.48,2.95,13.57,50.16-44.08,43.40
7,8,KFUM Oslo,30.0,11.67,7.42,10.91,44.46-36.25,42.43
8,9,Fredrikstad,30.0,11.28,8.10,10.62,36.65-34.87,41.93
9,10,Sarpsborg 08,30.0,9.70,9.98,10.32,46.65-44.34,39.08


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Bodø/Glimt,72.00,24.33,2.67,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Viking,26.33,64.33,7.33,1.67,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Brann,1.33,7.00,50.33,28.00,12.00,0.67,0.67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Tromsø,0.33,3.33,31.33,41.67,15.33,4.67,2.33,0.67,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Rosenborg,0.00,1.00,7.00,21.33,35.33,17.67,10.67,3.67,1.67,1.67,0.00,0.00,0.00,0.00,0.00,0.0
Molde,0.00,0.00,0.67,1.67,13.33,22.67,17.67,17.00,11.33,9.33,4.00,2.00,0.00,0.33,0.00,0.0
Sandefjord,0.00,0.00,0.67,2.33,10.33,21.33,20.00,15.33,13.67,9.33,4.67,1.33,1.00,0.00,0.00,0.0
KFUM Oslo,0.00,0.00,0.00,1.33,7.67,12.67,21.67,19.33,20.00,11.67,3.67,1.33,0.33,0.33,0.00,0.0
Fredrikstad,0.00,0.00,0.00,0.67,4.33,11.67,19.00,19.33,21.33,15.33,5.33,2.00,0.67,0.33,0.00,0.0
Sarpsborg 08,0.00,0.00,0.00,0.33,1.33,8.00,6.33,18.00,14.67,20.33,18.33,10.33,2.00,0.33,0.00,0.0


Median

In [ ]:
display(table_median_season)

# Playground

In [ ]:
fixtures.loc[
    ((fixtures['home'] == 'Sandefjord') & (fixtures['away'] == 'Viking')) & (fixtures['season'] == 2025)
]

In [ ]:
fixtures.loc[fixtures['id'] == 1342321, 'status'] = 'NS'

In [ ]:
fixtures.loc[((fixtures['home'] == 'Viking') | (fixtures['away'] == 'Viking')) & (fixtures['season'] == 2025)]